In [1]:
import numpy as np
import ipywidgets as widgets
import ipyvolume as ipv
from IPython.display import display
import pyuff

In [2]:
path="C:\\Users\\Lovro\\Documents\\5.Letnik\\Magistrska\\Podatki in meritve\\Tree_structure.uff"
file=pyuff.UFF(path)
sets = file.get_set_types()
sup_sets = file.get_supported_sets()

uffdict = {}
for a in sup_sets:
    index = []
    for i in range(len(sets)):
        if str(int(sets[i])) == a:
            index.append(i)
    uffdict[a] = index

def cleanup(dic):
    re_keys = []
    for key in dic.keys():
        if dic[key] == []:
            re_keys.append(key)
    for key in re_keys:
        del dic[key]
    return dic

uffdict=cleanup(uffdict)

keys_58 = ['0', '1', '2', '3', '4', '6']
keys_55 = ['2', '3', '5', '7']
dict58 = {'0':[], '1':[], '2':[], '3':[], '4':[], '6':[]}
dict55 = {'2':[], '3':[], '5':[], '7':[]}
nodes55 = {'2':[], '3':[], '5':[], '7':[]}
nodes58 = {'0':[], '1':[], '2':[], '3':[], '4':[], '6':[]}
for i in uffdict['58']:
    node = file.read_sets(i)['ref_node'] #lahko bi bil tudi resp_node, če bi bil rezultati določeni v točkah odziva
    f_type = file.read_sets(i)['func_type']
    if set(str(f_type)).issubset(set(keys_58)):
       dict58[str(f_type)].append(i)
    if set([node]).issubset(nodes58[str(f_type)])==False:
        if set([float(node)]).issubset(file.read_sets(uffdict['15'])['node_nums']):
            nodes58[str(f_type)].append(node)
for i in uffdict['55']:
    node = file.read_sets(i)['node_nums'] #lahko bi bil tudi resp_node, če bi bil rezultati določeni v točkah odziva
    d_type = file.read_sets(i)['data_type']
    if set(str(d_type)).issubset(set(keys_55)):
       dict55[str(d_type)].append(i)
    for n in node:
        if set([n]).issubset(nodes55[str(d_type)])==False:
            if set([float(n)]).issubset(file.read_sets(uffdict['15'])['node_nums']):
                nodes55[str(d_type)].append(int(n))
nodes55=cleanup(nodes55)
nodes58=cleanup(nodes58)

In [3]:
def viz (file,uffdict,data,dinfo=None):
    
    names55 ={'2': 'normal mode',
              '3': 'complex eigenvalue first order (displacement)',
              '5': 'frequency response',
              '7': 'complex eigenvalue second order (velocity)'}
    names58 ={'0': 'General or Unknown',
              '1': 'Time Response',
              '2': 'Auto Spectrum',
              '3': 'Cross Spectrum',
              '4': 'Frequency Response Function',
              '6': 'complex eigenvalue second order (velocity)'}
    in_names55 = {'normal mode':'2',
              'complex eigenvalue first order (displacement)':'3',
              'frequency response':'5',
              'complex eigenvalue second order (velocity)':'7'}
    in_names58 ={'General or Unknown':'0',
              'Time Response':'1',
              'Auto Spectrum':'2',
              'Cross Spectrum':'3',
              'Frequency Response Function':'4',
              'complex eigenvalue second order (velocity)':'6'}    
    
    x = np.asarray(file.read_sets(uffdict['15'])['x'])
    y = np.asarray(file.read_sets(uffdict['15'])['y'])
    z = np.asarray(file.read_sets(uffdict['15'])['z'])
    
    def s(x,y,z):
        ipv.figure()
        f = [ipv.scatter(x, y, z, size=2, marker='sphere',color='red')]
        if set(['82']).issubset(set(uffdict.keys())):
            pairs=[]
            for i in uffdict['82']:
                for j in range(1,file.read_sets(i)['n_nodes']):
                    pairs.append([int(file.read_sets(i)['nodes'][j-1]),int(file.read_sets(i)['nodes'][j])])
        f.append(ipv.plot_trisurf(x,y,z,lines=pairs))
        ipv.xyzlim(min(np.array([x,y,z]).flatten()),max(np.array([x,y,z]).flatten()))
        return f
    
    if type(data)==dict:
        buttons = widgets.RadioButtons(options=['Function data', 'Analysis'],description='Results type:')
        drop = widgets.Dropdown(options=[names58[key] for key in data['58'].keys()])
        s(x,y,z)
        def drop_data(*args):
            if buttons.value == 'Analysis':
                drop.options = [names55[key] for key in data['55'].keys()]
            if buttons.value=='Function data':
                drop.options = [names58[key] for key in data['58'].keys()]
        buttons.observe(drop_data,'value')
        a = ipv.scatter(np.array([0.]),np.array([0.]),np.array([0.]),size=3, marker='circle_2d',color='blue')
        def f(buttons,drop):
            if buttons=='Analysis':
                dset='55'
                a.x=x[data[dset][in_names55[drop]]]
                a.y=y[data[dset][in_names55[drop]]]
                a.z=z[data[dset][in_names55[drop]]]
                a.color='blue'
            if buttons=='Function data':
                dset='58'
                a.x=x[data[dset][in_names58[drop]]]
                a.y=y[data[dset][in_names58[drop]]]
                a.z=z[data[dset][in_names58[drop]]]
                a.color='green'
            
        widgets.interactive_output(f,{'buttons':buttons,'drop':drop})
        display(widgets.HBox([ipv.gcf(),widgets.VBox([buttons,drop])]))
        return buttons,drop
    
    if type(data)==np.ndarray:
        if data.ndim==3:#problems for more than 100 animating points
            if set(['dt']).issubset(list(dinfo.keys())):
                X = np.array([x+data[0,:,i] for i in range(data.shape[2])])
                Y = np.array([y+data[1,:,i] for i in range(data.shape[2])])
                Z = np.array([z+data[2,:,i] for i in range(data.shape[2])])
                anim = s(X,Y,Z)
                ipv.animation_control(anim,interval=dinfo['dt']*1000)
                ipv.show()

        if data.ndim==4:
            X = np.transpose(np.array([x[i]+data[0,i,:,:] for i in range(data.shape[1])]),(1,2,0))
            Y = np.transpose(np.array([y[i]+data[1,i,:,:] for i in range(data.shape[1])]),(1,2,0))
            Z = np.transpose(np.array([z[i]+data[2,i,:,:] for i in range(data.shape[1])]),(1,2,0))
            
            if dinfo['dset']==58:
                df=dinfo['df']
                def show_freq(freq):
                    f=int(freq/df-1)
                    anim = s(X[f],Y[f],Z[f])
                    ipv.animation_control(anim)
                    ipv.show()
                freq = widgets.BoundedIntText(min=0*df,max=(len(data[0,0,:,0])-1)*df,step=df,description='Hz')
                out1 = widgets.interactive_output(show_freq, {'freq':freq})
                display(widgets.HBox([out1,widgets.VBox([widgets.Label('Insert frequence by increment: %f Hz'%(df)),freq])]))
            
            if dinfo['dset']==55:
                mfreq=dinfo['freq']
                def norm_freq(freq):
                    f=mfreq.index(freq)
                    anim = s(X[f],Y[f],Z[f])
                    ipv.animation_control(anim)
                    ipv.show()
                def mshape(freq):
                    m=mfreq.index(freq)+1
                    display(widgets.Label('Mode shape: %i'%(m)))                
                freq = widgets.Dropdown(options=mfreq)
                out1 = widgets.interactive_output(mshape, {'freq':freq})
                out2 = widgets.interactive_output(norm_freq, {'freq':freq})
                display(widgets.HBox([widgets.VBox([out1,out2]),widgets.VBox([widgets.Label('Chose normal freqence'),widgets.HBox([freq,widgets.Label('Hz')])])]))
        else:
            print('error')

In [4]:
#data = {'55': nodes55,'58': nodes58}
data=np.random.rand(3,44,100)
data = np.append(data,-data,axis=2)
data = np.reshape(data,(3,44,100,2),order='F')

In [5]:
viz(file,uffdict,data,dinfo={'dset':55,'freq':[0,1,2,3,4,5,6,7,8,9]})